In [11]:
import torch as t
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
from jaxtyping import Float
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
import pandas as pd
from typing import Union
from einops import repeat
from huggingface_hub import snapshot_download
from huggingface_hub.utils import disable_progress_bars


from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from model_utils import *
from group_utils import *
from irrep_bounds import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os, sys
# HOME = os.environ['HOME']  # change if necessary
HOME = '/workspace/'
sys.path.append(f'{HOME}/Finite-groups/src')

In [61]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

def get_models(model_dir):
    disable_progress_bars()
    local_dir = f'{HOME}/models/{model_dir}'
    if not os.path.exists(local_dir):
        snapshot_download(repo_id=f'wiwu2390/{model_dir}', local_dir=local_dir)
    models, params = load_models(local_dir, sel='final')
    models = models[0]
    data = GroupData(params)
    group = data.groups[0]
    return models, data, group

MODEL0_DIR = '2024-12-31_01-57-44_S4_MLP2_32_ubias_wd2e-5_frac1'
MODEL1_DIR = '2024-12-31_02-07-29_S4_to_S4_A4x2_MLP2_32_ubias_wd2e-5_frac1'
model0, data0, group = get_models(MODEL0_DIR)
model1, data1, _ = get_models(MODEL1_DIR)
# select instance 5
# https://wandb.ai/neural_fate/group%20generalization/runs/len6zqlg/panel/un93bwhwa
model0 = model0[5]
model1 = model1[5]   
# model0 = model0[45]
# model1 = model1[45]   

Intersection size: 576/576 (1.00)
Added 576 elements from intersection
Added 0 elements from group 0: S(4)
Train set size: 576/576 (1.00)
Intersection size: 336/576 (0.58)
Added 336 elements from intersection
Added 0 elements from group 0: S(4)
Added 0 elements from group 1: times(A(4),Z(2))
Train set size: 336/576 (0.58)


/workspace/wilson/Finite-groups/src/model_utils.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(t.load(model_path, map_location=device))


In [62]:
for name, p0 in model0.named_parameters():
    p1 = model1.get_parameter(name)
    print(name, ((t.norm(p0 - p1)**2) / (t.norm(p0)**2)).item())

embedding_left 0.4111334979534149
embedding_right 0.42162278294563293
linear_left 0.7070237994194031
linear_right 0.7144994735717773
unembedding 0.6914891004562378
unembed_bias 0.48266756534576416


In [63]:
irreps0, irrep_idx_dict0 = get_neuron_irreps(model0, group)
irreps1, irrep_idx_dict1 = get_neuron_irreps(model1, group)

In [64]:
for k in sorted(set(irreps0.keys()).union(set(irreps1.keys()))):
    neurons0 = irrep_idx_dict0.get(k, [])
    neurons1 = irrep_idx_dict1.get(k, [])
    print(k, '|',  len(neurons0), len(neurons1), len(set(neurons0).intersection(neurons1)))

1d-0 | 0 0 0
1d-1 | 3 3 3
2d-0 | 2 0 0
3d-0 | 25 27 23
3d-1 | 2 0 0


In [60]:
irrep_idx_dict0

{'1d-0': [],
 '1d-1': [],
 '2d-0': [3, 10, 11, 18, 27],
 '3d-0': [0,
  1,
  2,
  4,
  5,
  6,
  7,
  9,
  12,
  16,
  17,
  19,
  20,
  21,
  22,
  23,
  25,
  26,
  28,
  29,
  30,
  31],
 '3d-1': [8, 14, 15, 24]}

In [49]:
irrep_idx_dict0['1d-1']

[]

In [44]:
loss_dict0 = test_loss(model0.to(device), data0)
loss_dict1 = test_loss(model1.to(device), data1)
loss_dict0, loss_dict1

({'G0_loss': tensor([0.0008], device='cuda:0'),
  'G0_acc': tensor([1.], device='cuda:0')},
 {'G0_loss': tensor([0.0078], device='cuda:0'),
  'G0_acc': tensor([1.], device='cuda:0'),
  'G1_loss': tensor([5.1937], device='cuda:0'),
  'G1_acc': tensor([0.5833], device='cuda:0')})